# Day8_2: 시계열 시각화 + 프로젝트 (Plotly 고급) - 정답

이 노트북은 **실습 퀴즈 정답**을 포함합니다.

---

## 사전 준비: 라이브러리 및 데이터

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta

---

## Q1. Range Slider 추가하기 ⭐

**문제**: 다음 매출 데이터에 Range Slider를 추가하고, "1개월", "3개월", "전체" 버튼을 만드세요.

In [ ]:
# Q1 정답

df_q1 = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=180, freq='D'),
    'revenue': 1000 + pd.Series(range(180)).apply(lambda x: x * 5 + (x % 30 - 15) * 10)
})

# 1. 기본 라인 차트 생성
fig = px.line(
    df_q1,
    x='date',
    y='revenue',
    title='매출 추이 (Range Slider 포함)',
    labels={'revenue': '매출 (만원)', 'date': '날짜'}
)

# 2. Range Slider 및 Range Selector 추가
fig.update_xaxes(
    rangeslider_visible=True,  # Range Slider 활성화
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1개월", step="month", stepmode="backward"),
            dict(count=3, label="3개월", step="month", stepmode="backward"),
            dict(step="all", label="전체")
        ])
    )
)

fig.show()

print("✅ Range Slider 및 Range Selector 추가 완료")

### 💡 풀이 설명

**접근 방법**:
- `update_xaxes()`로 X축에 Range Slider와 Range Selector를 추가
- `rangeslider_visible=True`로 하단 슬라이더 활성화
- `rangeselector`의 `buttons`에 기간별 버튼 정의

**핵심 개념**:
- `count`: 숫자 (1, 3 등)
- `step`: 단위 ("month", "year", "day")
- `stepmode`: "backward" (현재부터 과거로), "todate" (년/월 시작부터)
- `step="all"`: 전체 데이터 표시

**대안 솔루션**:
```python
# 더 많은 버튼 추가
dict(count=1, label="1주일", step="week", stepmode="backward"),
dict(count=6, label="6개월", step="month", stepmode="backward"),
```

**흔한 실수**:
- ❌ `rangeselector`를 `update_layout()`에 추가 → `update_xaxes()`가 맞음
- ❌ `stepmode` 누락 → 기본값이 적용되지만 명시적으로 지정 권장

**실무 팁**:
1년 이상 데이터는 Range Slider 필수! 사용자 탐색 편의성이 크게 향상됩니다.

---

## Q2. Annotation 추가하기 ⭐⭐

**문제**: 주가 데이터에서 최고가와 최저가 지점에 Annotation을 추가하세요.

In [ ]:
# Q2 정답

df_q2 = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=60, freq='D'),
    'price': [100, 102, 105, 103, 108, 110, 107, 112, 115, 113] * 6
})

# 1. 최고가/최저가 인덱스 찾기
max_idx = df_q2['price'].idxmax()
min_idx = df_q2['price'].idxmin()

max_price = df_q2.loc[max_idx, 'price']
min_price = df_q2.loc[min_idx, 'price']
max_date = df_q2.loc[max_idx, 'date']
min_date = df_q2.loc[min_idx, 'date']

# 2. 기본 차트 생성
fig = px.line(df_q2, x='date', y='price', title='주가 차트 with 최고가/최저가 Annotations')

# 3. 최고가 Annotation
fig.add_annotation(
    x=max_date,
    y=max_price,
    text=f"📈 최고가: ${max_price}",
    showarrow=True,
    arrowhead=2,
    arrowcolor='red',
    ax=0,
    ay=-40,
    font=dict(size=12, color='red', family='Arial Black')
)

# 4. 최저가 Annotation
fig.add_annotation(
    x=min_date,
    y=min_price,
    text=f"📉 최저가: ${min_price}",
    showarrow=True,
    arrowhead=2,
    arrowcolor='blue',
    ax=0,
    ay=40,
    font=dict(size=12, color='blue', family='Arial Black')
)

fig.show()

print(f"최고가: ${max_price} ({max_date.date()})")
print(f"최저가: ${min_price} ({min_date.date()})")

In [ ]:
# 테스트 케이스로 검증
assert max_price == df_q2['price'].max(), "최고가 계산 오류"
assert min_price == df_q2['price'].min(), "최저가 계산 오류"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. `idxmax()`, `idxmin()`으로 최대/최소값의 인덱스 찾기
2. `.loc[]`로 해당 행의 날짜와 가격 추출
3. `add_annotation()`으로 화살표와 텍스트 추가

**핵심 개념**:
- `ax`, `ay`: 화살표 끝점의 상대 위치 (픽셀)
- `ay=-40`: 위쪽으로 40픽셀 (최고가)
- `ay=40`: 아래쪽으로 40픽셀 (최저가)
- `arrowhead`: 화살표 스타일 (0~8)

**대안 솔루션**:
```python
# 수직선으로 표시
fig.add_vline(
    x=max_date,
    line_dash="dash",
    line_color="red",
    annotation_text=f"최고가: ${max_price}"
)
```

**흔한 실수**:
- ❌ `idxmax()` 대신 `max()` 사용 → 값만 반환, 인덱스 없음
- ❌ `ax`, `ay` 부호 반대 → 화살표 방향 잘못됨

**실무 팁**:
중요 이벤트는 Annotation으로 강조! 발표 자료나 리포트에서 시선을 끕니다.

---

## Q3. 다중 Y축 차트 만들기 ⭐⭐

**문제**: 매출(만원)과 이익률(%)을 다중 Y축으로 표시하세요.

In [ ]:
# Q3 정답

df_q3 = pd.DataFrame({
    'month': pd.date_range('2024-01-01', periods=12, freq='ME'),
    'revenue': [5000, 5200, 5800, 6100, 6500, 7000, 7200, 7500, 7800, 8000, 8500, 9000],
    'profit_rate': [10, 12, 14, 13, 15, 16, 14, 17, 18, 16, 19, 20]
})

# 1. Figure 생성 (graph_objects 사용)
fig = go.Figure()

# 2. 매출 (막대 그래프, 왼쪽 Y축)
fig.add_trace(go.Bar(
    x=df_q3['month'],
    y=df_q3['revenue'],
    name='매출',
    marker_color='#95E1D3',
    yaxis='y1'
))

# 3. 이익률 (선 그래프, 오른쪽 Y축)
fig.add_trace(go.Scatter(
    x=df_q3['month'],
    y=df_q3['profit_rate'],
    name='이익률',
    line=dict(color='#F38181', width=3),
    mode='lines+markers',
    marker=dict(size=10),
    yaxis='y2'
))

# 4. 레이아웃 설정 (다중 Y축)
fig.update_layout(
    title='월별 매출 및 이익률 분석',
    xaxis=dict(title='월'),
    yaxis=dict(
        title='매출 (만원)',
        side='left',
        showgrid=True
    ),
    yaxis2=dict(
        title='이익률 (%)',
        side='right',
        overlaying='y',  # y1 위에 겹침
        showgrid=False
    ),
    hovermode='x unified',
    height=500
)

fig.show()

print(f"평균 매출: {df_q3['revenue'].mean():,.0f}만원")
print(f"평균 이익률: {df_q3['profit_rate'].mean():.1f}%")

In [ ]:
# 테스트 케이스로 검증
assert len(fig.data) == 2, "2개의 trace가 있어야 함"
assert fig.data[0].yaxis == 'y1', "첫 번째 trace는 y1 축"
assert fig.data[1].yaxis == 'y2', "두 번째 trace는 y2 축"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. `go.Figure()`로 빈 캔버스 생성
2. `add_trace()`로 각 그래프 추가, `yaxis` 파라미터로 축 지정
3. `update_layout()`에서 `yaxis2`를 `overlaying='y'`로 설정

**핵심 개념**:
- `yaxis='y1'`: 첫 번째 Y축 (기본, 왼쪽)
- `yaxis='y2'`: 두 번째 Y축 (오른쪽)
- `overlaying='y'`: y1 위에 y2를 겹침
- `side='left'` or `'right'`: 축 위치

**대안 솔루션**:
```python
# make_subplots 사용 (더 복잡하지만 유연)
from plotly.subplots import make_subplots
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(...), secondary_y=False)
fig.add_trace(go.Scatter(...), secondary_y=True)
```

**흔한 실수**:
- ❌ `yaxis2`에 `overlaying='y'` 누락 → 별도 축으로 표시됨
- ❌ `showgrid=True` 양쪽에 설정 → 그리드 선 겹침으로 혼란

**실무 팁**:
단위가 크게 다른 지표만 다중 Y축 사용! 비슷한 스케일은 단일 축이 더 명확합니다.

---

## Q4. 이동평균 계산하기 ⭐⭐⭐

**문제**: 다음 주가 데이터에서 5일, 20일, 60일 이동평균을 계산하고 한 차트에 표시하세요.

In [ ]:
# Q4 정답

df_q4 = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=100, freq='D'),
    'price': 100 + np.random.randn(100).cumsum() * 2
})

# 1. 이동평균 계산
df_q4['MA5'] = df_q4['price'].rolling(window=5).mean()
df_q4['MA20'] = df_q4['price'].rolling(window=20).mean()
df_q4['MA60'] = df_q4['price'].rolling(window=60).mean()

# 2. 시각화
fig = go.Figure()

# 주가 (회색, 얇게)
fig.add_trace(go.Scatter(
    x=df_q4['date'],
    y=df_q4['price'],
    name='주가',
    line=dict(color='lightgray', width=1),
    opacity=0.5
))

# MA5 (빨강)
fig.add_trace(go.Scatter(
    x=df_q4['date'],
    y=df_q4['MA5'],
    name='MA5 (단기)',
    line=dict(color='red', width=2)
))

# MA20 (파랑)
fig.add_trace(go.Scatter(
    x=df_q4['date'],
    y=df_q4['MA20'],
    name='MA20 (중기)',
    line=dict(color='blue', width=2)
))

# MA60 (초록)
fig.add_trace(go.Scatter(
    x=df_q4['date'],
    y=df_q4['MA60'],
    name='MA60 (장기)',
    line=dict(color='green', width=2)
))

fig.update_layout(
    title='주가 및 이동평균선 차트',
    xaxis_title='날짜',
    yaxis_title='가격 ($)',
    hovermode='x unified',
    height=500
)

fig.show()

print(f"현재가: ${df_q4['price'].iloc[-1]:.2f}")
print(f"MA5: ${df_q4['MA5'].iloc[-1]:.2f}")
print(f"MA20: ${df_q4['MA20'].iloc[-1]:.2f}")
print(f"MA60: ${df_q4['MA60'].iloc[-1]:.2f}")

In [ ]:
# 테스트 케이스로 검증
assert 'MA5' in df_q4.columns, "MA5 컬럼 누락"
assert 'MA20' in df_q4.columns, "MA20 컬럼 누락"
assert 'MA60' in df_q4.columns, "MA60 컬럼 누락"
assert df_q4['MA5'].notna().sum() == 100 - 5 + 1, "MA5 계산 오류"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. `rolling(window=N).mean()`으로 N일 이동평균 계산
2. 주가를 회색 얇은 선으로, 이평선을 굵은 색상 선으로 구분
3. `hovermode='x unified'`로 날짜별 모든 값 표시

**핵심 개념**:
- `rolling(window=N)`: N개 데이터의 이동 윈도우
- `.mean()`: 윈도우 내 평균 계산
- 처음 N-1개는 NaN (데이터 부족)
- MA5 < MA20 < MA60: 단기일수록 가격 변화에 빠르게 반응

**대안 솔루션**:
```python
# pandas-ta 라이브러리 사용
import pandas_ta as ta
df_q4['MA5'] = ta.sma(df_q4['price'], length=5)
```

**흔한 실수**:
- ❌ `.mean()` 대신 `.sum()` 사용 → 평균이 아닌 합계
- ❌ NaN 처리 없이 계산 → 초반 값 누락

**실무 팁**:
이동평균은 추세 파악의 기본! 단기/중기/장기 조합으로 매매 신호 생성합니다.

---

## Q5. Candlestick 차트 생성 ⭐⭐⭐

**문제**: 다음 OHLC 데이터로 Candlestick 차트를 만드세요.

In [ ]:
# Q5 정답

df_q5 = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=30, freq='D'),
    'open': [100, 102, 101, 105, 107] * 6,
    'high': [105, 107, 106, 110, 112] * 6,
    'low': [98, 100, 99, 103, 105] * 6,
    'close': [103, 101, 104, 108, 106] * 6
})

# Candlestick 차트 생성
fig = go.Figure(data=[go.Candlestick(
    x=df_q5['date'],
    open=df_q5['open'],
    high=df_q5['high'],
    low=df_q5['low'],
    close=df_q5['close'],
    name='OHLC',
    increasing_line_color='#26A69A',  # 상승 캔들 색상
    decreasing_line_color='#EF5350'   # 하락 캔들 색상
)])

fig.update_layout(
    title='Candlestick 차트',
    xaxis_title='날짜',
    yaxis_title='가격 ($)',
    xaxis_rangeslider_visible=False,  # 하단 슬라이더 숨김
    height=500
)

fig.show()

print(f"최고가: ${df_q5['high'].max()}")
print(f"최저가: ${df_q5['low'].min()}")
print(f"상승일: {(df_q5['close'] > df_q5['open']).sum()}일")
print(f"하락일: {(df_q5['close'] < df_q5['open']).sum()}일")

In [ ]:
# 테스트 케이스로 검증
assert fig.data[0].type == 'candlestick', "Candlestick 타입이 아님"
assert len(fig.data[0].x) == 30, "데이터 개수 오류"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. `go.Candlestick()`에 OHLC 데이터 전달
2. `increasing_line_color`, `decreasing_line_color`로 색상 커스터마이징
3. `xaxis_rangeslider_visible=False`로 하단 슬라이더 제거 (옵션)

**핵심 개념**:
- **Open**: 시가 (장 시작 가격)
- **High**: 고가 (최고 가격)
- **Low**: 저가 (최저 가격)
- **Close**: 종가 (장 마감 가격)
- **초록 캔들**: Close > Open (상승)
- **빨강 캔들**: Close < Open (하락)

**대안 솔루션**:
```python
# plotly.express는 Candlestick 미지원, graph_objects만 가능
```

**흔한 실수**:
- ❌ OHLC 순서 바꿈 → 차트 깨짐
- ❌ 컬럼명 대소문자 오류 → `'Open'` vs `'open'`

**실무 팁**:
Candlestick은 금융 데이터 필수! 한눈에 가격 움직임과 변동성을 파악할 수 있습니다.

---

## Q6. 골든/데드 크로스 탐지 ⭐⭐⭐⭐

**문제**: Q4의 데이터에서 MA5와 MA20의 골든 크로스와 데드 크로스를 탐지하고, 차트에 마커로 표시하세요.

In [ ]:
# Q6 정답

# 1. 신호 생성
df_q4['signal'] = 0
df_q4.loc[df_q4['MA5'] > df_q4['MA20'], 'signal'] = 1   # 골든 크로스 구간
df_q4.loc[df_q4['MA5'] < df_q4['MA20'], 'signal'] = -1  # 데드 크로스 구간

# 2. 크로스 발생 지점 찾기 (신호 변화)
df_q4['signal_change'] = df_q4['signal'].diff()
golden_cross = df_q4[df_q4['signal_change'] == 2]  # -1 → 1 (데드 → 골든)
dead_cross = df_q4[df_q4['signal_change'] == -2]   # 1 → -1 (골든 → 데드)

print(f"골든 크로스 발생: {len(golden_cross)}회")
print(f"데드 크로스 발생: {len(dead_cross)}회")

# 3. 시각화
fig = go.Figure()

# 주가
fig.add_trace(go.Scatter(
    x=df_q4['date'], y=df_q4['price'],
    name='주가', line=dict(color='lightgray', width=1), opacity=0.5
))

# MA5
fig.add_trace(go.Scatter(
    x=df_q4['date'], y=df_q4['MA5'],
    name='MA5', line=dict(color='red', width=2)
))

# MA20
fig.add_trace(go.Scatter(
    x=df_q4['date'], y=df_q4['MA20'],
    name='MA20', line=dict(color='blue', width=2)
))

# 골든 크로스 마커
if len(golden_cross) > 0:
    fig.add_trace(go.Scatter(
        x=golden_cross['date'],
        y=golden_cross['price'],
        mode='markers',
        name='골든 크로스 (매수)',
        marker=dict(
            symbol='triangle-up',
            size=15,
            color='green',
            line=dict(width=2, color='white')
        )
    ))

# 데드 크로스 마커
if len(dead_cross) > 0:
    fig.add_trace(go.Scatter(
        x=dead_cross['date'],
        y=dead_cross['price'],
        mode='markers',
        name='데드 크로스 (매도)',
        marker=dict(
            symbol='triangle-down',
            size=15,
            color='red',
            line=dict(width=2, color='white')
        )
    ))

fig.update_layout(
    title='골든 크로스 & 데드 크로스 탐지',
    xaxis_title='날짜',
    yaxis_title='가격 ($)',
    hovermode='x unified',
    height=500
)

fig.show()

In [ ]:
# 테스트 케이스로 검증
assert 'signal' in df_q4.columns, "signal 컬럼 누락"
assert 'signal_change' in df_q4.columns, "signal_change 컬럼 누락"
assert df_q4['signal'].isin([0, 1, -1]).all(), "signal 값 오류"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. `signal` 컬럼 생성: MA5 > MA20이면 1, MA5 < MA20이면 -1
2. `diff()`로 신호 변화 계산: -1 → 1이면 골든, 1 → -1이면 데드
3. 크로스 지점만 필터링하여 마커로 표시

**핵심 개념**:
- **골든 크로스**: 단기 MA가 장기 MA를 상향 돌파 → 상승 추세 전환 신호
- **데드 크로스**: 단기 MA가 장기 MA를 하향 돌파 → 하락 추세 전환 신호
- `signal_change == 2`: -1 → 1 (차이가 2)
- `signal_change == -2`: 1 → -1 (차이가 -2)

**대안 솔루션**:
```python
# numpy를 활용한 교차점 탐지
cross_over = (df_q4['MA5'] > df_q4['MA20']) & (df_q4['MA5'].shift(1) <= df_q4['MA20'].shift(1))
cross_under = (df_q4['MA5'] < df_q4['MA20']) & (df_q4['MA5'].shift(1) >= df_q4['MA20'].shift(1))
```

**흔한 실수**:
- ❌ `signal_change == 1` 사용 → 0 → 1 변화도 포함됨
- ❌ NaN 처리 없이 계산 → 초반 크로스 누락

**실무 팁**:
골든/데드 크로스는 매매 타이밍의 기본! 하지만 과거 데이터로 백테스팅 필수입니다.

---

## Q7. 볼린저 밴드 계산 ⭐⭐⭐⭐

**문제**: Q4의 데이터에 볼린저 밴드(20일, 2σ)를 계산하고 시각화하세요.

In [ ]:
# Q7 정답

# 1. 볼린저 밴드 계산
window = 20
num_std = 2

df_q4['BB_Middle'] = df_q4['price'].rolling(window=window).mean()
df_q4['BB_Std'] = df_q4['price'].rolling(window=window).std()
df_q4['BB_Upper'] = df_q4['BB_Middle'] + (df_q4['BB_Std'] * num_std)
df_q4['BB_Lower'] = df_q4['BB_Middle'] - (df_q4['BB_Std'] * num_std)

# 2. 시각화
fig = go.Figure()

# 상단 밴드 (빨강 점선)
fig.add_trace(go.Scatter(
    x=df_q4['date'],
    y=df_q4['BB_Upper'],
    name='상단 밴드',
    line=dict(color='red', width=1, dash='dash'),
    opacity=0.5
))

# 중간 밴드 (파랑 실선)
fig.add_trace(go.Scatter(
    x=df_q4['date'],
    y=df_q4['BB_Middle'],
    name='중간 밴드 (MA20)',
    line=dict(color='blue', width=2)
))

# 하단 밴드 (초록 점선, 상단과 사이 채우기)
fig.add_trace(go.Scatter(
    x=df_q4['date'],
    y=df_q4['BB_Lower'],
    name='하단 밴드',
    line=dict(color='green', width=1, dash='dash'),
    fill='tonexty',  # 이전 trace(상단 밴드)와의 사이 채우기
    fillcolor='rgba(100,100,100,0.1)',
    opacity=0.5
))

# 주가 (검은색 실선)
fig.add_trace(go.Scatter(
    x=df_q4['date'],
    y=df_q4['price'],
    name='주가',
    line=dict(color='black', width=2)
))

fig.update_layout(
    title='볼린저 밴드 (20일, 2σ)',
    xaxis_title='날짜',
    yaxis_title='가격 ($)',
    hovermode='x unified',
    height=500
)

fig.show()

print(f"밴드 폭 (최근): ${df_q4['BB_Upper'].iloc[-1] - df_q4['BB_Lower'].iloc[-1]:.2f}")
print(f"현재가 위치: {((df_q4['price'].iloc[-1] - df_q4['BB_Lower'].iloc[-1]) / (df_q4['BB_Upper'].iloc[-1] - df_q4['BB_Lower'].iloc[-1]) * 100):.1f}%")

In [ ]:
# 테스트 케이스로 검증
assert 'BB_Upper' in df_q4.columns, "BB_Upper 컬럼 누락"
assert 'BB_Lower' in df_q4.columns, "BB_Lower 컬럼 누락"
assert (df_q4['BB_Upper'] >= df_q4['BB_Middle']).all(), "상단 밴드가 중간보다 낮음"
assert (df_q4['BB_Middle'] >= df_q4['BB_Lower']).all(), "중간 밴드가 하단보다 낮음"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. `rolling(window).mean()`으로 중간 밴드 (이동평균) 계산
2. `rolling(window).std()`로 표준편차 계산
3. 상단 = 중간 + 2σ, 하단 = 중간 - 2σ
4. `fill='tonexty'`로 상단~하단 사이 채우기

**핵심 개념**:
- **중간 밴드**: 20일 이동평균 (추세)
- **상단 밴드**: 중간 + 2×표준편차 (과매수 경계)
- **하단 밴드**: 중간 - 2×표준편차 (과매도 경계)
- **밴드 폭**: 변동성 지표 (좁으면 큰 움직임 예고)

**대안 솔루션**:
```python
# pandas-ta 라이브러리 사용
import pandas_ta as ta
bb = ta.bbands(df_q4['price'], length=20, std=2)
df_q4 = pd.concat([df_q4, bb], axis=1)
```

**흔한 실수**:
- ❌ `std()` 대신 `var()` 사용 → 분산 값으로 계산됨
- ❌ `fill='tonexty'` 순서 오류 → 상단 먼저 추가해야 함

**실무 팁**:
볼린저 밴드 밴드 폭이 좁아지면 변동성 확대 직전! 큰 가격 움직임을 예측하는 신호입니다.

---

## Q8. yfinance 데이터 수집 ⭐⭐⭐⭐

**문제**: yfinance로 **테슬라(TSLA)** 주식의 **최근 6개월** 데이터를 수집하고, Candlestick 차트를 만드세요.

In [ ]:
# Q8 정답

# 1. 데이터 수집
print("📡 테슬라 데이터 수집 중...")
tsla = yf.Ticker("TSLA")
df_tsla = tsla.history(period="6mo")

print(f"✅ 데이터 수집 완료: {len(df_tsla)}일치")
print(f"기간: {df_tsla.index.min().date()} ~ {df_tsla.index.max().date()}")

# 2. Candlestick 차트
fig = go.Figure(data=[go.Candlestick(
    x=df_tsla.index,
    open=df_tsla['Open'],
    high=df_tsla['High'],
    low=df_tsla['Low'],
    close=df_tsla['Close'],
    name='TSLA',
    increasing_line_color='#26A69A',
    decreasing_line_color='#EF5350'
)])

# 3. Range Slider 추가
fig.update_layout(
    title='테슬라(TSLA) 최근 6개월 주가',
    xaxis_title='날짜',
    yaxis_title='가격 ($)',
    xaxis_rangeslider_visible=True,  # Range Slider 활성화
    height=600
)

fig.show()

# 4. 주요 통계
print(f"\n📊 테슬라 주가 통계:")
print(f"최고가: ${df_tsla['High'].max():.2f}")
print(f"최저가: ${df_tsla['Low'].min():.2f}")
print(f"현재가: ${df_tsla['Close'].iloc[-1]:.2f}")
print(f"평균 거래량: {df_tsla['Volume'].mean():,.0f}")

In [ ]:
# 테스트 케이스로 검증
assert len(df_tsla) > 0, "데이터 수집 실패"
assert 'Open' in df_tsla.columns, "Open 컬럼 누락"
assert 'High' in df_tsla.columns, "High 컬럼 누락"
assert 'Low' in df_tsla.columns, "Low 컬럼 누락"
assert 'Close' in df_tsla.columns, "Close 컬럼 누락"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. `yf.Ticker("TSLA")`로 티커 객체 생성
2. `.history(period="6mo")`로 6개월 데이터 다운로드
3. `go.Candlestick()`으로 OHLC 차트 생성
4. Range Slider 추가로 탐색 편의성 향상

**핵심 개념**:
- **yfinance**: Yahoo Finance API 래퍼, 무료 금융 데이터 제공
- **period**: "1d", "5d", "1mo", "3mo", "6mo", "1y", "2y", "5y", "10y", "ytd", "max"
- **interval**: "1m", "2m", "5m", "15m", "30m", "60m", "90m", "1h", "1d", "5d", "1wk", "1mo", "3mo"

**대안 솔루션**:
```python
# 특정 날짜 범위 지정
df_tsla = tsla.history(start="2023-01-01", end="2024-01-01")
```

**흔한 실수**:
- ❌ 티커 심볼 오류 (예: "TESLA" → "TSLA")
- ❌ 네트워크 오류 처리 없음 → try-except 권장
- ❌ API 제한 무시 → 과도한 요청 시 차단 가능

**실무 팁**:
yfinance는 무료지만 데이터 정확성 보장 없음! 실전 트레이딩은 유료 API 사용 권장합니다.

---

## Q9. 다중 지표 통합 차트 ⭐⭐⭐⭐⭐

**문제**: Q8의 테슬라 데이터에 다음 요소를 모두 포함하는 차트를 만드세요.

요구사항:
1. Candlestick 차트
2. MA7, MA21 이동평균선
3. 볼린저 밴드 (20일, 2σ)
4. 골든 크로스 / 데드 크로스 마커
5. 거래량 (하단 별도 차트)

In [ ]:
# Q9 정답

# 1. 기술적 지표 계산
df_tsla = df_tsla.reset_index()

# 이동평균
df_tsla['MA7'] = df_tsla['Close'].rolling(window=7).mean()
df_tsla['MA21'] = df_tsla['Close'].rolling(window=21).mean()

# 볼린저 밴드
df_tsla['BB_Middle'] = df_tsla['Close'].rolling(window=20).mean()
df_tsla['BB_Std'] = df_tsla['Close'].rolling(window=20).std()
df_tsla['BB_Upper'] = df_tsla['BB_Middle'] + (df_tsla['BB_Std'] * 2)
df_tsla['BB_Lower'] = df_tsla['BB_Middle'] - (df_tsla['BB_Std'] * 2)

# 골든/데드 크로스
df_tsla['signal'] = 0
df_tsla.loc[df_tsla['MA7'] > df_tsla['MA21'], 'signal'] = 1
df_tsla.loc[df_tsla['MA7'] < df_tsla['MA21'], 'signal'] = -1
df_tsla['signal_change'] = df_tsla['signal'].diff()

golden_cross = df_tsla[df_tsla['signal_change'] == 2]
dead_cross = df_tsla[df_tsla['signal_change'] == -2]

print(f"골든 크로스: {len(golden_cross)}회")
print(f"데드 크로스: {len(dead_cross)}회")

# 2. 서브플롯 생성 (2행: 가격 + 거래량)
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    row_heights=[0.7, 0.3],
    subplot_titles=('테슬라(TSLA) 종합 차트', '거래량')
)

# === 상단: 가격 차트 ===

# Candlestick
fig.add_trace(go.Candlestick(
    x=df_tsla['Date'],
    open=df_tsla['Open'],
    high=df_tsla['High'],
    low=df_tsla['Low'],
    close=df_tsla['Close'],
    name='TSLA',
    increasing_line_color='#26A69A',
    decreasing_line_color='#EF5350'
), row=1, col=1)

# MA7
fig.add_trace(go.Scatter(
    x=df_tsla['Date'], y=df_tsla['MA7'],
    name='MA7', line=dict(color='orange', width=2)
), row=1, col=1)

# MA21
fig.add_trace(go.Scatter(
    x=df_tsla['Date'], y=df_tsla['MA21'],
    name='MA21', line=dict(color='blue', width=2)
), row=1, col=1)

# 볼린저 밴드 상단
fig.add_trace(go.Scatter(
    x=df_tsla['Date'], y=df_tsla['BB_Upper'],
    name='BB 상단', line=dict(color='red', dash='dot', width=1), opacity=0.5
), row=1, col=1)

# 볼린저 밴드 하단
fig.add_trace(go.Scatter(
    x=df_tsla['Date'], y=df_tsla['BB_Lower'],
    name='BB 하단', line=dict(color='green', dash='dot', width=1),
    fill='tonexty', fillcolor='rgba(150,150,150,0.1)', opacity=0.5
), row=1, col=1)

# 골든 크로스
if len(golden_cross) > 0:
    fig.add_trace(go.Scatter(
        x=golden_cross['Date'],
        y=golden_cross['Close'],
        mode='markers',
        name='골든 크로스',
        marker=dict(symbol='triangle-up', size=15, color='green', line=dict(width=2, color='white'))
    ), row=1, col=1)

# 데드 크로스
if len(dead_cross) > 0:
    fig.add_trace(go.Scatter(
        x=dead_cross['Date'],
        y=dead_cross['Close'],
        mode='markers',
        name='데드 크로스',
        marker=dict(symbol='triangle-down', size=15, color='red', line=dict(width=2, color='white'))
    ), row=1, col=1)

# === 하단: 거래량 ===
colors = ['green' if close > open_ else 'red' 
          for close, open_ in zip(df_tsla['Close'], df_tsla['Open'])]

fig.add_trace(go.Bar(
    x=df_tsla['Date'],
    y=df_tsla['Volume'],
    name='거래량',
    marker_color=colors,
    opacity=0.5
), row=2, col=1)

# 레이아웃
fig.update_layout(
    title='테슬라(TSLA) 종합 분석 대시보드',
    xaxis2_title='날짜',
    yaxis_title='가격 ($)',
    yaxis2_title='거래량',
    hovermode='x unified',
    height=800,
    showlegend=True,
    xaxis_rangeslider_visible=False
)

fig.show()

# 3. 주요 통계
print(f"\n📊 테슬라 분석 리포트")
print(f"기간: {df_tsla['Date'].min().date()} ~ {df_tsla['Date'].max().date()}")
print(f"최고가: ${df_tsla['High'].max():.2f}")
print(f"최저가: ${df_tsla['Low'].min():.2f}")
print(f"현재가: ${df_tsla['Close'].iloc[-1]:.2f}")
print(f"MA7: ${df_tsla['MA7'].iloc[-1]:.2f}")
print(f"MA21: ${df_tsla['MA21'].iloc[-1]:.2f}")

In [ ]:
# 테스트 케이스로 검증
assert len(fig.data) >= 7, "최소 7개 trace 필요 (Candlestick, MA2, BB2, Volume, 크로스)"
assert 'MA7' in df_tsla.columns, "MA7 누락"
assert 'BB_Upper' in df_tsla.columns, "BB_Upper 누락"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. 모든 지표 계산 (MA, BB, 크로스)
2. `make_subplots()`로 2행 레이아웃 생성
3. `row`, `col` 파라미터로 각 trace를 적절한 위치에 배치
4. 거래량 색상을 상승/하락에 따라 동적 설정

**핵심 개념**:
- `make_subplots()`: 복잡한 레이아웃 생성 도구
- `shared_xaxes=True`: X축 공유로 연동 줌/팬
- `row_heights`: 각 행의 비율 조정
- `subplot_titles`: 각 차트 제목

**대안 솔루션**:
```python
# 3×1 레이아웃 (가격, 거래량, RSI)
fig = make_subplots(
    rows=3, cols=1,
    row_heights=[0.5, 0.25, 0.25]
)
```

**흔한 실수**:
- ❌ `row`, `col` 누락 → 모든 trace가 같은 위치에 겹침
- ❌ `shared_xaxes=False` → X축 독립적으로 움직임
- ❌ 거래량 색상 하드코딩 → 상승/하락 구분 없음

**실무 팁**:
종합 대시보드는 한눈에 모든 정보 파악! 프로젝트 발표나 리포트에 강력한 시각 자료입니다.

---

## Q10. 암호화폐 비교 대시보드 ⭐⭐⭐⭐⭐

**문제**: **비트코인(BTC-USD)**과 **이더리움(ETH-USD)**의 최근 3개월 데이터를 수집하고, **2×2 서브플롯**으로 비교 대시보드를 만드세요.

In [ ]:
# Q10 정답

# 1. 데이터 수집
print("📡 암호화폐 데이터 수집 중...")

btc = yf.Ticker("BTC-USD")
eth = yf.Ticker("ETH-USD")

df_btc = btc.history(period="3mo").reset_index()
df_eth = eth.history(period="3mo").reset_index()

print(f"✅ BTC 데이터: {len(df_btc)}일")
print(f"✅ ETH 데이터: {len(df_eth)}일")

# 2. 이동평균 계산
df_btc['MA7'] = df_btc['Close'].rolling(window=7).mean()
df_btc['MA21'] = df_btc['Close'].rolling(window=21).mean()

df_eth['MA7'] = df_eth['Close'].rolling(window=7).mean()
df_eth['MA21'] = df_eth['Close'].rolling(window=21).mean()

# 3. 2×2 서브플롯 생성
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'BTC 가격 차트',
        'ETH 가격 차트',
        'BTC 거래량',
        'ETH 거래량'
    ),
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

# === (1,1): BTC Candlestick + MA ===
fig.add_trace(go.Candlestick(
    x=df_btc['Date'],
    open=df_btc['Open'],
    high=df_btc['High'],
    low=df_btc['Low'],
    close=df_btc['Close'],
    name='BTC',
    increasing_line_color='#26A69A',
    decreasing_line_color='#EF5350'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df_btc['Date'], y=df_btc['MA7'],
    name='BTC MA7', line=dict(color='orange', width=2)
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df_btc['Date'], y=df_btc['MA21'],
    name='BTC MA21', line=dict(color='blue', width=2)
), row=1, col=1)

# === (1,2): ETH Candlestick + MA ===
fig.add_trace(go.Candlestick(
    x=df_eth['Date'],
    open=df_eth['Open'],
    high=df_eth['High'],
    low=df_eth['Low'],
    close=df_eth['Close'],
    name='ETH',
    increasing_line_color='#26A69A',
    decreasing_line_color='#EF5350'
), row=1, col=2)

fig.add_trace(go.Scatter(
    x=df_eth['Date'], y=df_eth['MA7'],
    name='ETH MA7', line=dict(color='orange', width=2)
), row=1, col=2)

fig.add_trace(go.Scatter(
    x=df_eth['Date'], y=df_eth['MA21'],
    name='ETH MA21', line=dict(color='blue', width=2)
), row=1, col=2)

# === (2,1): BTC 거래량 ===
btc_colors = ['green' if c > o else 'red' for c, o in zip(df_btc['Close'], df_btc['Open'])]
fig.add_trace(go.Bar(
    x=df_btc['Date'],
    y=df_btc['Volume'],
    name='BTC Volume',
    marker_color=btc_colors,
    opacity=0.5
), row=2, col=1)

# === (2,2): ETH 거래량 ===
eth_colors = ['green' if c > o else 'red' for c, o in zip(df_eth['Close'], df_eth['Open'])]
fig.add_trace(go.Bar(
    x=df_eth['Date'],
    y=df_eth['Volume'],
    name='ETH Volume',
    marker_color=eth_colors,
    opacity=0.5
), row=2, col=2)

# 레이아웃
fig.update_layout(
    title='🪙 암호화폐 비교 대시보드 (BTC vs ETH)',
    height=800,
    showlegend=True,
    hovermode='x unified'
)

# X축 Range Slider 숨김
fig.update_xaxes(rangeslider_visible=False)

fig.show()

# 4. 주요 통계 비교
print("\n" + "="*60)
print("📊 암호화폐 비교 리포트")
print("="*60)

print(f"\n🪙 비트코인 (BTC):")
print(f"  최고가: ${df_btc['High'].max():,.2f}")
print(f"  최저가: ${df_btc['Low'].min():,.2f}")
print(f"  현재가: ${df_btc['Close'].iloc[-1]:,.2f}")
print(f"  평균 거래량: {df_btc['Volume'].mean():,.0f}")

print(f"\n🪙 이더리움 (ETH):")
print(f"  최고가: ${df_eth['High'].max():,.2f}")
print(f"  최저가: ${df_eth['Low'].min():,.2f}")
print(f"  현재가: ${df_eth['Close'].iloc[-1]:,.2f}")
print(f"  평균 거래량: {df_eth['Volume'].mean():,.0f}")

print("\n" + "="*60)

In [ ]:
# 테스트 케이스로 검증
assert len(df_btc) > 0, "BTC 데이터 수집 실패"
assert len(df_eth) > 0, "ETH 데이터 수집 실패"
assert len(fig.data) == 8, "8개의 trace 필요 (BTC: Candlestick+MA2+Volume, ETH: Candlestick+MA2+Volume)"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. BTC, ETH 데이터 각각 수집
2. `make_subplots(rows=2, cols=2)`로 2×2 그리드 생성
3. `row`, `col` 파라미터로 각 차트 위치 지정
4. 거래량 색상을 상승/하락에 따라 동적 생성

**핵심 개념**:
- `make_subplots(rows=2, cols=2)`: 2×2 그리드
- `subplot_titles`: 각 차트 제목 튜플
- `vertical_spacing`, `horizontal_spacing`: 차트 간 간격
- `update_xaxes(rangeslider_visible=False)`: 모든 X축에 적용

**대안 솔루션**:
```python
# 1×2 레이아웃 (좌우 비교)
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"secondary_y": True}, {"secondary_y": True}]]
)
# 각 차트에 거래량을 secondary Y축으로 추가
```

**흔한 실수**:
- ❌ `subplot_titles` 개수 불일치 → 에러 발생
- ❌ `row`, `col` 잘못된 인덱스 → 차트 위치 오류
- ❌ 모든 trace에 `showlegend=True` → 범례 중복

**실무 팁**:
다중 자산 비교는 투자 포트폴리오 분석 필수! 상관관계와 리스크 분산을 시각적으로 파악합니다.